In [1]:
## PATH settings
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(project_root)
COMMON_PATH = os.path.join(project_root, 'common')
from common.extractor.alpaca_extractor import AlpacaExtractor
from common.processor.utils import Utils
from datetime import timedelta
from piboufilings import get_filings
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import requests as r
from common.database.adatabase import ADatabase

In [2]:
alp = AlpacaExtractor(paper=False)

In [3]:
ams = pd.read_csv("asset_management.csv",delimiter="\t").dropna()
ams["CIK"] = [str((int(x[:-2]))) for x in ams["CIK"].astype(str)]
ams["CIK"] = ams["CIK"].astype(str).str.zfill(10)
holdings = []
for cik in tqdm(ams["CIK"].unique()):
    try:
        holding = list(get_filings(
            cik=cik, #
            form_type=["13F-HR"],
            start_year=2025, 
            end_year = 2026,
            user_agent="chung.ejy@gmail.com"
        ).values())[0]
        holding["cik"] = cik
        holdings.append(holding)
    except Exception as e:
        print(str(e))

 14%|██████████████████████████████▌                                                                                                                                                                                       | 1/7 [00:05<00:31,  5.23s/it]

cannot access local variable 'all_parsed_dataframes' where it is not associated with a value



 43%|███████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                                          | 3/7 [00:20<00:27,  6.90s/it]

cannot access local variable 'all_parsed_dataframes' where it is not associated with a value


 57%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                           | 4/7 [00:26<00:19,  6.35s/it]

...

cannot access local variable 'all_parsed_dataframes' where it is not associated with a value


 86%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                              | 6/7 [00:37<00:05,  5.90s/it]

cannot access local variable 'all_parsed_dataframes' where it is not associated with a value


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:43<00:00,  6.23s/it]

cannot access local variable 'all_parsed_dataframes' where it is not associated with a value


In [4]:
a = pd.concat(holdings)

In [5]:
a.columns

Index(['ACCESSION_NUMBER', 'CONFORMED_DATE', 'NAME_OF_ISSUER',
       'TITLE_OF_CLASS', 'CUSIP', 'SHARE_VALUE', 'SHARE_AMOUNT', 'SH_PRN',
       'PUT_CALL', 'DISCRETION', 'SOLE_VOTING_AUTHORITY',
       'SHARED_VOTING_AUTHORITY', 'NONE_VOTING_AUTHORITY', 'CREATED_AT',
       'UPDATED_AT', 'cik'],
      dtype='object')

In [18]:
current = a[a["CONFORMED_DATE"]==a["CONFORMED_DATE"].max()]
print(current["CONFORMED_DATE"].max())
securities = current[["NAME_OF_ISSUER","TITLE_OF_CLASS","SHARE_VALUE","SHARE_AMOUNT"]].groupby(["NAME_OF_ISSUER","TITLE_OF_CLASS"]).mean().reset_index()

2025-03-31 00:00:00


In [7]:
securities["share_price"] = securities["SHARE_VALUE"] / securities["SHARE_AMOUNT"]
securities["weight"] = securities["SHARE_VALUE"] / securities["SHARE_VALUE"].sum()

In [8]:
def get_ticker(company_name):
    yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    params = {"q": company_name, "quotes_count": 1, "country": "United States"}

    res = r.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
    data = res.json()

    company_code = data['quotes'][0]['symbol']
    return company_code

In [9]:
tickers = []
for company in tqdm(securities["NAME_OF_ISSUER"]):
    try:
        cleaned = " ".join(company.split(" ")[:-1])
        results = get_ticker(cleaned)
        tickers.append({"NAME_OF_ISSUER":company,"ticker":results})
    except:
        tickers.append({"NAME_OF_ISSUER":company,"ticker":None})

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4280/4280 [14:08<00:00,  5.04it/s]


In [10]:
tickers_df = pd.DataFrame(tickers)
ticker_dict = tickers_df.set_index("NAME_OF_ISSUER")["ticker"].to_dict()

In [11]:
securities["ticker"] = securities["NAME_OF_ISSUER"].apply(lambda x: ticker_dict[x])

In [12]:
securities

,NAME_OF_ISSUER,TITLE_OF_CLASS,SHARE_VALUE,SHARE_AMOUNT,share_price,weight,ticker
0,1 800 FLOWERS COM INC,CL A,1947804.8,330136.4,5.9,0.000002,None
1,10X GENOMICS INC,CL A COM,19412178.0,2223617.2,8.73,0.000021,TXG
2,1ST SOURCE CORP,COM,12776540.4,213618.8,59.81,0.000014,SRCE
3,1STDIBS COM INC,COM,1524723.0,501553.666667,3.04,0.000002,None
4,2SEVENTY BIO INC,COMMON STOCK,4642725.666667,939823.0,4.94,0.000005,TSVT
...,...,...,...,...,...,...,...
4275,ZUMIEZ INC,COM,5445893.666667,365741.666667,14.890001,0.000006,ZUMZ
4276,ZURA BIO LTD,CLASS A ORD SHS,362128.0,280719.333333,1.29,0.0,ZURA
4277,ZURN ELKAY WATER SOLNS CORP,COM,104618720.6,3172186.8,32.98,0.000111,None
4278,ZYMEWORKS INC,COM,4209091.6,353408.2,11.91,0.000004,ZYME


In [13]:
securities["cs"] = ["COM" in x for x in securities["TITLE_OF_CLASS"]]

In [16]:
okie = securities[securities["cs"]==True][["NAME_OF_ISSUER","ticker","TITLE_OF_CLASS","share_price","SHARE_VALUE"]]

In [17]:
ori = ADatabase("ori")
ori.connect()
ori.drop("ams")
ori.store("ams",okie)
ori.disconnect()